# 1) Données

In [1]:
import pandas as pd
import numpy as np
import csv
import os

# calcul de la position dans le roulement d'un poste en fonction de la semaine
def rFunction(s,p):
    return ((p-s)%24)

# calcul du numéro de semaine correspondant au jour j
def sFunction(j):
    return j//7

# calcul du jour non travaillé pour une semaine donnée d'un opérateur à 80%
# en fonction du jour non travaillé la première semaine
def delta_i(s,d_i):
    return (d_i+s)%5

# operateurs + competences
dataOperateurs = pd.read_csv('./data/operateurs.csv',sep = ";" )

cardO = len(dataOperateurs.axes[0])
colsDataOperateurs = len(dataOperateurs.axes[1])
nomsOperateurs = dataOperateurs.iloc[0:cardO,0:3]
competencesOperateurs = dataOperateurs.iloc[0:cardO,3:(colsDataOperateurs+1)]

# postes (intitulés + rho)
dataPostes = pd.read_csv('./data/postes.csv',sep = ";" )
cardP = len(dataPostes.axes[0])
nomsPostes = dataPostes.iloc[0:cardO,1]
rho = [0,1,1,1,0,1,1,1,0,1,1,1,0,1,1,1,0,1,1,1,0,1,1,1]

# le nombre de semaines <=> nombre de postes <=> nombre d'opérateurs
#cardS = cardO
cardS = 15
cardJ = 7*cardS
cardR = cardO

# kappa_ip (opérateur possède la compétence pour le poste)
#kappa = pd.read_csv('./data/kappa_ip.csv',sep = ";" )
kappa = [
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,0,1,1,1,0,0,0],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,0,0,1,1,1,0,1,1,1,1,1,1,0,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,0,1,1,1,0,0,0],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,0,1,1,1,0,0,0],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,0,1,1,1,0,0,0]]

# sigma_pp' (postes sur le même créneau horaire)
#sigma = pd.read_csv('./data/sigma_pp.csv',sep = ";" )
sigma= [[1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
[0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1],
[0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
[0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1],
[0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0],
[0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1]]

d = [3,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1]

# o_j (jour ouvrés)
o = np.zeros(cardJ)
for j in range(0,len(o)):
    if (j%7 <= 4):
        o[j] = 1

# H_s (jours de chaque semaine)
H = []
for i in range(0,cardS):
    H.append(np.zeros(7))
for i in range(0,cardS):
    for j in range(0,7):
        H[i][j]=(i*7 + j)
        
profils = np.zeros(cardO)
cardProfils = 0
for i in range(0,cardO):
    found = False
    for j in range(0,i):
        if not found:
            diff = False
            if d[i] == d[j]:
                for p in range(0,cardP):
                    if kappa[i][p] != kappa[j][p]:
                        diff = True
            else:
                diff = True
        
            if not diff:
                profil = profils[j]
                found = True
    if found:
        profils[i] = profil
    else:
        profils[i] = cardProfils
        cardProfils = cardProfils + 1
        
        
print(profils)
print(cardProfils)
N = np.zeros(cardProfils)
for i in range(cardO):
    N[int(profils[i])] += 1
print(N)

# delta_ij (est-ce qu'une personne à 80% travaille le jour j)
#delta = pd.read_csv('./data/delta_ij.csv',sep = ";" )
# d[i] représente le jour non travaillé de l'opérateur i la première semaine, d[i]=-1 si i est à temps plein

delta = np.full((cardProfils, cardJ), 1)
for profil in range(cardProfils):
    found = False
    i = 0
    index = -1
    while not found and i < cardO:
        if profils[i] == profil:
            found = True
            index = i
        i += 1
    if d[index] != -1:
        d_i = d[index]
        for s in range(0, cardS):
            d_is = delta_i(s,d_i) #rang du jour non travaillé la semaine s
            j_s = 7*s + d_is      #numéro du jour non travaillé
            delta[profil][j_s] = 0
            
kappaProfils = []
for profil in range(cardProfils):
    found = False
    i = 0
    index = -1
    while not found and i < cardO:
        if profils[i] == profil:
            found = True
            index = i
        i += 1
    kappaProfils.append(kappa[index])
    
print(kappaProfils)
    
    

print(delta[0])

#print(nomsOperateurs)
#print(competencesOperateurs)
#print(nomsPostes)
#print(rho)
#print(kappa)
#print(sigma)
#print(delta)
#print(o)
#print(H[1])


[0. 1. 2. 1. 1. 2. 3. 1. 2. 1. 4. 1. 1. 1. 1. 1. 2. 3. 1. 3. 2. 1. 1. 3.]
5
[ 1. 13.  5.  4.  1.]
[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0], [1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1]]
[1 1 1 0 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1
 1 0 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0
 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1]


# 2) Modèle mathématique


## 2.1) Variables

In [2]:
from pulp import *

# x_ijp = 1 si l'opérateur prend en charge le poste p le jour j
x = {} 
for prof in range(0,cardProfils):
    for j in range(0,cardJ): 
        for p in range(0,cardP): 
            x[prof,j,p]=LpVariable(f"x({prof,j,p})", cat=LpBinary) #LpBinary

# y_ir = 1 si l'opérateur i est position r dans le roulement
y = {} 
for prof in range(0,cardProfils):
    for r in range(0,cardR):    
        y[prof,r]=LpVariable(f"y({prof,r})", cat=LpBinary)
        
# z_isp = 1 si l'opérateur est affecté au poste p la semaine s
z = {} 
for prof in range(0,cardProfils):
    for s in range(0,cardS): 
        for p in range(0,cardP): 
            z[prof,s,p]=LpVariable(f"z({prof,s,p})", cat=LpBinary)

# u_is = 1 si l'opérateur i est affecté à un poste différent de celui prévu la semaine s
u = {} 
for p in range(0,cardP):
    for s in range(0,cardS):    
        u[prof,s]=LpVariable(f"u({prof,s})", cat=LpBinary)

print("nb de variables x_ijp = ", len(x))
print("nb de variables y_ir  = ", len(y))
print("nb de variables z_isp = ", len(z))
print("nb de variables u_ir  = ", len(u))
print()
print("nb total de variables = ", len(x)+len(y)+len(z)+len(u))

nb de variables x_ijp =  12600
nb de variables y_ir  =  120
nb de variables z_isp =  1800
nb de variables u_ir  =  75

nb total de variables =  14595


## 2.2) Création du modèle avec PuLP

In [3]:
prob = LpProblem("ModelPlanning", LpMinimize)

prob += sum(u[prof,s] for prof in range(0,cardProfils) for s in range(0,cardS)), "obj"

## 2.2) Contraintes

In [4]:
#contrainte 1 Un opérateur ne peut prendre un poste que s'il est compétent et disponible sur un jour ouvré

for prof in range(0,cardProfils):
    for j in range(0,cardJ):
        for p in range(0,cardP):
            prob += (x[prof,j,p] <= delta[prof][j]*kappaProfils[prof][p]*o[j], f"C1_{prof,j,p}")

#contrainte 2 Un opérateur ne peut être affecté à un poste une semaine que s'il en possède la compétence

for prof in range(0,cardProfils):
    for s in range(0,cardS):
        for p in range(0,cardP):
            prob += (z[prof,s,p] <= kappaProfils[prof][p], f"C2_{prof,s,p}")

#contrainte 3 Chaque jour, l'opérateur est affecté à son poste de la semaine (s'il n'est pas rouleur)
for prof in range(0,cardProfils):
    for s in range(0,cardS):
        for j in H[s]:
            for p in range(0,cardP):
                prob += x[prof,j,p] <= z[prof,s,p] + sum((1 - rho[p2])*z[prof,s,p2] for p2 in range(0,cardP)), f"C3_{prof,j,p}"

#contrainte 4 (page 8) Chaque jour, un opérateur est affecté à au plus un poste #MODIF

for prof in range(0,cardProfils):
    for j in range(0,cardJ):
        if o[j] == 1 and delta[prof][j] == 1:
            prob += sum(x[prof,j,p] for p in range(0,cardP)) == N[prof], f"C4_{prof,j,p}"

#contrainte 5 (page 8) Chaque jour, chaque poste est assuré par un opérateur #MODIF
for p in range(0,cardP):
    for j in range(0,cardJ):
        if o[j] == 1 and rho[p] == 1:
            prob += sum(x[prof,j,p] for prof in range(0,cardProfils)) == 1, f"C5_{prof,j,p}"
            
#contrainte 5.2 BONUS Chaque jour, un poste roulant est occupé par au plus une personne
for p in range(0,cardP):
    for j in range(0,cardJ):
        if o[j] == 1 and rho[p] == 0:
            prob += sum(x[prof,j,p] for prof in range(0,cardProfils)) <= 1, f"C52_{prof,j,p}"

#contrainte 6 (page 9) Chaque semaine, chaque opérateur est affecté à un poste

for prof in range(0,cardProfils):
    for s in range(0,cardS):
        prob += sum(z[prof,s,p] for p in range(0,cardP)) == N[prof], f"C6_{prof,s,p}"

#contrainte 7 (page 9) Chaque semaine, chaque poste est assuré par un opérateur

for p in range(0,cardP):
    for s in range(0,cardS):
        prob += sum(z[prof,s,p] for prof in range(0,cardProfils)) == 1, f"C7_{prof,s,p}"

#contrainte 8 (page 10) Chaque opérateur est affecté à une position dans le roulement

for prof in range(0,cardProfils):
    prob += sum(y[prof,r] for r in range(0,cardR)) == N[prof], f"C8_{prof,r}"

#contrainte 9 (page 10) Chaque position dans le roulement est associée à un opérateur

for r in range(0,cardR):
    prob += sum(y[prof,r] for prof in range(0,cardProfils)) == 1, f"C9_{prof,r}"

#contrainte 10 (page 11) Un opérateur peut changer de poste si l'horaire de ce dernier est compatible

for prof in range(0,cardProfils):
    for s in range(0,cardS):
        for p in range(0,cardP):
            prob += z[prof,s,p] <= sum(sigma[p][p2]*y[prof,rFunction(s,p2)] for p2 in range(0,cardP)), f"C10_{prof,s,p}"

#contrainte 11 (page 11) Lorsqu'un jour un opérateur est affecté à un poste A alors qu'il est affecté à un poste B pour cette semaine, cela augmente son score d'insatisfaction
for prof in range(0,cardProfils):
    for s in range(0,cardS):
        for p in range(0,cardP):
            prob += u[p,s] >= (kappaProfils[prof][p]*rho[p]*y[prof,rFunction(s,p)] - z[prof,s,p]), f"C11_{p,s}"


## 2.4) Résolution du problème

In [5]:
import time

prob.writeLP("Model.lp")

start = time.time() 
prob.solve(CPLEX_PY())
end = time.time()

duree = end - start
print("Temps de résolution : " + str(duree))

Version identifier: 22.1.0.0 | 2022-03-09 | 1a383f8ce
CPXPARAM_Read_DataCheck                          1
Tried aggregator 2 times.
MIP Presolve eliminated 19965 rows and 4950 columns.
MIP Presolve modified 2160 coefficients.
Aggregator did 15 substitutions.
Reduced MIP has 13244 rows, 9630 columns, and 93480 nonzeros.
Reduced MIP has 9630 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.09 sec. (71.67 ticks)
Found incumbent of value 75.000000 after 0.22 sec. (143.89 ticks)
Probing changed sense of 450 constraints.
Probing time = 0.14 sec. (89.45 ticks)
Tried aggregator 1 time.
Detecting symmetries...
MIP Presolve eliminated 540 rows and 450 columns.
Reduced MIP has 12704 rows, 9180 columns, and 92130 nonzeros.
Reduced MIP has 9180 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.22 sec. (140.65 ticks)
Probing time = 0.02 sec. (8.95 ticks)
Clique table members: 51172.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
P

   1803  1288        0.4270  1559        2.0000        0.3491  1555385   82.54%
   1870  1370        0.6262  1144        2.0000        0.3491  1636114   82.54%
   1929  1423        0.6213   925        2.0000        0.3503  1701691   82.49%
   1988  1447        cutoff              2.0000        0.3504  1723019   82.48%
Elapsed time = 152.07 sec. (63511.92 ticks, tree = 3.35 MB, solutions = 16)
   2054  1483        0.4261  1268        2.0000        0.3504  1768869   82.48%
   2141  1591        0.5726   793        2.0000        0.3511  1890363   82.44%
   2202  1588        0.6895  1024        2.0000        0.3533  1882188   82.34%
   2272  1691        0.7086   908        2.0000        0.3534  2016898   82.33%
   2353  1665        0.9830   597        2.0000        0.3544  1986895   82.28%
   2431  1820        0.5250  1129        2.0000        0.3544  2136784   82.28%
   2486  1827        0.3935  1019        2.0000        0.3553  2144371   82.23%
   2537  1931        0.4109  1444        2.0

Elapsed time = 442.60 sec. (184383.49 ticks, tree = 5.93 MB, solutions = 16)
   7338  2755        0.6394  1399        2.0000        0.4546  6993748   77.27%
   7564  2939        0.9512  1088        2.0000        0.4572  7209529   77.14%
   7780  3089        0.7952  1453        2.0000        0.4579  7436220   77.10%
   7998  3233        0.5542  1750        2.0000        0.4597  7641182   77.01%
   8168  3332        0.6567  1484        2.0000        0.4625  7767523   76.88%
   8342  3492        0.7554  1309        2.0000        0.4639  8015059   76.81%
   8585  3678        0.9703  1029        2.0000        0.4652  8248028   76.74%
   8832  3738        0.5756  1159        2.0000        0.4661  8313841   76.69%
   9082  3934        0.7576  1184        2.0000        0.4692  8552491   76.54%
   9273  4090        0.9120  1559        2.0000        0.4712  8801031   76.44%
Elapsed time = 540.72 sec. (222707.12 ticks, tree = 10.33 MB, solutions = 16)
   9486  4239        0.6601  1452        2.00

  26976 12014        cutoff              2.0000        0.5982 25116887   70.09%
  27247 12099        cutoff              2.0000        0.5989 25465193   70.05%
  27519 12156        0.9360  1262        2.0000        0.5994 25558004   70.03%
  27766 12294        cutoff              2.0000        0.6007 25849644   69.96%
  27976 12428        cutoff              2.0000        0.6019 26145384   69.90%
  28190 12413        0.7625  1300        2.0000        0.6025 26105215   69.88%
  28355 12504        0.9276  1311        2.0000        0.6043 26395258   69.78%
Elapsed time = 7455.78 sec. (567863.70 ticks, tree = 31.71 MB, solutions = 16)
  28523 12622        cutoff              2.0000        0.6062 26671810   69.69%
  28686 12670        0.8029   986        2.0000        0.6080 26849565   69.60%
  28844 12777        0.6534  1494        2.0000        0.6101 27318361   69.50%
  29049 12775        cutoff              2.0000        0.6106 27291227   69.47%
  29289 12787        0.7034  1388        

  48821 15889        cutoff              2.0000        0.7058 43806733   64.71%
  49144 15885        0.8139  1239        2.0000        0.7065 43955365   64.67%
  49442 15920        0.9192  1203        2.0000        0.7081 44199056   64.59%
Elapsed time = 8412.43 sec. (913076.19 ticks, tree = 40.39 MB, solutions = 16)
  49706 15949        0.7987  1345        2.0000        0.7088 44569081   64.56%
  49965 15933        cutoff              2.0000        0.7117 44681194   64.41%
  50160 15929        0.7847  1072        2.0000        0.7117 44921079   64.41%
  50341 15958        cutoff              2.0000        0.7140 45759169   64.30%
  50538 15961        0.9140  1299        2.0000        0.7148 45864885   64.26%
  50754 15990        0.9379  1466        2.0000        0.7167 46037827   64.16%
  50983 15948        0.8348  1434        2.0000        0.7167 45912949   64.16%
  51247 15956        0.9239  1120        2.0000        0.7180 45844731   64.10%
  51549 15971        0.7743  1150        

  75167 10928        0.9338  1268        2.0000        0.8283 63119887   58.58%
  75451 11198        0.9784   526        2.0000        0.8290 62821965   58.55%
  75754 10688        cutoff              2.0000        0.8303 63402588   58.48%
  76048 10828        cutoff              2.0000        0.8320 63280125   58.40%
  76373 10731        0.9631  1161        2.0000        0.8352 63364244   58.24%
  76712 10390        0.9700  1493        2.0000        0.8365 63802282   58.18%
  77027 10095        cutoff              2.0000        0.8384 64227612   58.08%
  77384 10266        0.9029  1481        2.0000        0.8402 64018229   57.99%
  77718 10198        cutoff              2.0000        0.8430 64106341   57.85%
  78068  9563        0.9019  1354        2.0000        0.8436 64724718   57.82%
Elapsed time = 9460.94 sec. (1296397.02 ticks, tree = 23.95 MB, solutions = 16)
  78462  9823        cutoff              2.0000        0.8458 64484771   57.71%
  78837  9153        cutoff             

## 2.5) Extraction de la solution

In [6]:
print("objectif =", value(prob.objective))

objectif = 2.0


## 2.6) Affichage de la solution

In [10]:
print("uis")
for s in range(0,cardS):
    for i in range(0,cardProfils):
        if(u[i,s].varValue != 0):
            print("u[",i,",",s,"] =", u[i,s].varValue)

print('Affectation aux roulements')
for i in range(cardProfils):
    for r in range(0,24):
        if(y[i,r].varValue != 0):
            print('y[',i,',',r,'] =', y[i,r].varValue)

print()
print('Affectations hebdomadaires')
for s in range(0,cardS):
    print('semaine',s)
    for i in range(0,cardProfils):
        for p in range(0,cardP):
            if z[i,s,p].varValue==1:
                print('semaine',s,',',i,'affecté au poste',p)
                
print('======================================================')

for p in range(0,cardP):
    print()
    if rho[p] == 1:
        print('poste',p,'non rouleur')
    else:
        print('poste',p,'rouleur')
    for j in range(0,cardJ):
        for i in range(0,cardProfils):
            if (x[i,j,p].varValue == 1):
                print(i,'occupe le poste',p,'le jour',j)
                
print('======================================================')

                
for i in range(0,cardO):
    print()
    print('planning de l\'opérateur',i)
    for j in range(0,cardJ):
        for p in range(0,cardP):
            if (x[i,j,p].varValue == 1):
                print('poste',p,'le jour',j)

uis
u[ 1 , 13 ] = 1.0
u[ 1 , 14 ] = 1.0
Affectation aux roulements
y[ 0 , 17 ] = 1.0
y[ 1 , 5 ] = 1.0
y[ 1 , 6 ] = 1.0
y[ 1 , 7 ] = 1.0
y[ 1 , 8 ] = 1.0
y[ 1 , 9 ] = 1.0
y[ 1 , 10 ] = 1.0
y[ 1 , 11 ] = 1.0
y[ 1 , 12 ] = 1.0
y[ 1 , 13 ] = 1.0
y[ 1 , 14 ] = 1.0
y[ 1 , 15 ] = 1.0
y[ 1 , 16 ] = 1.0
y[ 1 , 20 ] = 1.0
y[ 2 , 3 ] = 1.0
y[ 2 , 4 ] = 1.0
y[ 2 , 19 ] = 1.0
y[ 2 , 21 ] = 1.0
y[ 2 , 22 ] = 1.0
y[ 3 , 0 ] = 1.0
y[ 3 , 1 ] = 1.0
y[ 3 , 2 ] = 1.0
y[ 3 , 23 ] = 1.0
y[ 4 , 18 ] = 1.0

Affectations hebdomadaires
semaine 0
semaine 0 , 0 affecté au poste 17
semaine 0 , 1 affecté au poste 5
semaine 0 , 1 affecté au poste 6
semaine 0 , 1 affecté au poste 7
semaine 0 , 1 affecté au poste 8
semaine 0 , 1 affecté au poste 9
semaine 0 , 1 affecté au poste 10
semaine 0 , 1 affecté au poste 11
semaine 0 , 1 affecté au poste 12
semaine 0 , 1 affecté au poste 13
semaine 0 , 1 affecté au poste 14
semaine 0 , 1 affecté au poste 15
semaine 0 , 1 affecté au poste 16
semaine 0 , 1 affecté au poste 20
se

KeyError: (5, 0, 0)

# Extraction des données au format CSV

In [ ]:
import csv
with open('res.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    jours = [""]
    for s in range(0,cardS):
        jours.extend(["lundi","mardi","mercredi","jeudi","vendredi","samedi","dimanche"])
    writer.writerow(jours)
    
    dates = [""]
    for j in range(0,cardJ):
        dates.append("jour "+str(j))
    writer.writerow(dates)
    
    for i in range(0,cardO):
        planningPerso = [nomsOperateurs.iloc[i,1] + " " + nomsOperateurs.iloc[i,2]]
        for j in range(0,cardJ):
            found = False
            for p in range(0,cardP):
                if x[i,j,p].varValue == 1:
                    found = True
                    planningPerso.append(nomsPostes.iloc[p])
            if not found:
                if o[j] == 0:
                    planningPerso.append("")
                else:
                    planningPerso.append("RA")
        writer.writerow(planningPerso)